# Lesson 3: Agentic Search

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


'The Blackwell GPU features enhanced Transformer Engine, Confidential Computing, and a 10 TB/s interconnect for superior AI performance. It includes 208 billion transistors and supports large-scale AI models. Blackwell is designed for generative AI and secure AI training.'

## Regular search

In [5]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
   
"""
#"weather.com"

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [6]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

returning previous results due to exception reaching ddg.
https://weather.com/weather/today/l/USCA0987:1:US
https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8


In [8]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [9]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

returning previous results due to exception reaching ddg.
Website: https://weather.com/weather/today/l/USCA0987:1:US


<body><div class="appWrapper DaybreakLargeScreen LargeScreen lightTheme twcTheme DaybreakLargeScreen--appWrapper--ZkDop" id="appWrapper"><div class="DaybreakLargeScreen--appContentWrapper--MdeHd appContentWrapper"><div class="region-meta"><div class="removeIfEmpty" id="WxuHtmlHead-meta-"></div><div class="removeIfEmpty" id="WxuNewsroom-meta-bc9f40d5-d941-4fd8-bae2-2d8d63a38bb3"></div></div><div class="regionHeaderWrapper DaybreakLargeScreen--stickyHeader--0Pgu8"><div class="regionHeaderInnerWrapper"><div class="adsSectionOuterWrapper"><div class="adsSectionInnerWrapper"><div class="stickyAdPlaceholder"></div><div class="js-branded-background-ads" id="labBG"></div><div class="js-branded-background-ads" id="wx-hero-content"></div><div class="region-stickyAds regionStickyAds"><div class="removeIfEmpty" id="WxuAd-stickyAds-425f392e-14ee-4059-8218-451bb2b284c0"><div class="

In [10]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/weather/today/l/USCA0987:1:US


Recent Locations Weather Forecasts Radar & Maps News & Media Products & Account Lifestyle Specialty Forecasts San Francisco, CA Weather Today in San Francisco, CA 6:47 am 7:24 pm Hourly Weather - San Francisco, CA Now Cloudy 11 pm Cloudy 12 am Mostly Cloudy 1 am Mostly Cloudy Don't Miss Seasonal Hub 10 Day Weather - San Francisco, CA Tonight Night Cloudy skies. Low near 60F. Winds WNW at 5 to 10 mph. Thu 11 Day Cloudy early with partial sunshine expected late. High 71F. Winds W at 10 to 20 mph. Night Partly cloudy early with increasing clouds overnight. Low 59F. Winds W at 10 to 15 mph. Fri 12 Day Mostly cloudy early, then afternoon sunshine. High near 70F. Winds WSW at 10 to 20 mph. Night A few clouds from time to time. Expect mist and reduced visibilities at times. Low 58F. Winds WSW at 10 to 20 mph. Sat 13 Day Mostly cloudy skies early will become partly cloudy later in the day. High around 70F. Winds WSW at 10 to 20 mph. 

## Agentic Search

In [11]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1757564548, 'localtime': '2025-09-10 21:22'}, 'current': {'last_updated_epoch': 1757564100, 'last_updated': '2025-09-10 21:15', 'temp_c': 18.9, 'temp_f': 66.0, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1014.0, 'pressure_in': 29.95, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 75, 'cloud': 100, 'feelslike_c': 18.9, 'feelslike_f': 66.0, 'windchill_c': 15.9, 'windchill_f': 60.7, 'heatindex_c': 15.9, 'heatindex_f': 60.7, 'dewpoint_c': 15.1, 'dewpoint_f': 59.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 14.1, 'gust_kph': 22.7}}


In [12]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "United States of America",
        "lat": 37.775,
        "lon": -122.4183,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1757564548,
        "localtime": "2025-09-10 21:22"
    },
    "current": {
        "last_updated_epoch": 1757564100,
        "last_updated": "2025-09-10 21:15",
        "temp_c": 18.9,
        "temp_f": 66.0,
        "is_day": 0,
        "condition": {
            "text": "Overcast",
            "icon": "//cdn.weatherapi.com/weather/64x64/night/122.png",
            "code": 1009
        },
        "wind_mph": 9.4,
        "wind_kph": 15.1,
        "wind_degree": 270,
        "wind_dir": "W",
        "pressure_mb": 1014.0,
        "pressure_in": 29.95,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 75,
        "cloud": 100,
        "feelslike_c": 18.9,
        "feelslike_f": 66.0,
        "windchill_c": 15.9,
        

<img src="./google_sample.png" width="800" height="600">